In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
from image_preprocessors import add_noise_random, cut_and_center
# Global settings
TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE =  128
CATEGORIES = 26
LETTER_END_POSITION = "P1"
IMAGE_PREPOCESSORS = [
    {"func" :cut_and_center, "params": ""}, 
    {"func": add_noise_random, "params": [0, 35]}
]

In [ ]:

%load_ext autoreload
%autoreload 2

from morse_helpers import create_sets
from morse_label_funcs import  labels_to_one_hot_positions_categorical, letter_n_to_index, position_regression

(image_fnames, morse_labels) = create_sets(
    [
        # ["./training_data/MorseTrainSet_04/GEN04_VER_000/", 'wordsMatrices_04_000', "Words_04_000.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_100/", 'wordsMatrices_04_100', "Words_04_100.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_200/", 'wordsMatrices_04_200', "Words_04_200.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_210/", 'wordsMatrices_04_210', "Words_04_210.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_220/", 'wordsMatrices_04_220', "Words_04_220.csv"],

        # ["./training_data/MorseTrainSet_11/GEN11_VER_000/", 'wordsMatrices_11_000', "Words_11_000.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_100/", 'wordsMatrices_11_100', "Words_11_100.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_200/", 'wordsMatrices_11_200', "Words_11_200.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_210/", 'wordsMatrices_11_210', "Words_11_210.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_220/", 'wordsMatrices_11_220', "Words_11_220.csv"],

        # ["./training_data/MorseTrainSet_06/GEN06_VER_000/", 'wordsMatrices_06_000', "Words_06_000.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_100/", 'wordsMatrices_06_100', "Words_06_100.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_200/", 'wordsMatrices_06_200', "Words_06_200.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_210/", 'wordsMatrices_06_210', "Words_06_210.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_220/", 'wordsMatrices_06_220', "Words_06_220.csv"],

        # ["./training_data/MorseTrainSet_13/GEN13_VER_000/", 'wordsMatrices_13_000', "Words_13_000.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_100/", 'wordsMatrices_13_100', "Words_13_100.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_200/", 'wordsMatrices_13_200', "Words_13_200.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_210/", 'wordsMatrices_13_210', "Words_13_210.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_220/", 'wordsMatrices_13_220', "Words_13_220.csv"],

        # ["./training_data/MorseTrainSet_15/GEN15_VER_000/", 'wordsMatrices_15_000', "Words_15_000.csv"],
        # ["./training_data/MorseTrainSet_15/GEN15_VER_100/", 'wordsMatrices_15_100', "Words_15_100.csv"],
        # ["./training_data/MorseTrainSet_15/GEN15_VER_200/", 'wordsMatrices_15_200', "Words_15_200.csv"],
        # ["./training_data/MorseTrainSet_15/GEN15_VER_210/", 'wordsMatrices_15_210', "Words_15_210.csv"],
        # ["./training_data/MorseTrainSet_15/GEN15_VER_220/", 'wordsMatrices_15_220', "Words_15_220.csv"],

        # ["./training_data/MorseTrainSet_15/GEN15_VER_001/", 'wordsMatrices_15_001', "Words_15_001.csv"],
        # ["./training_data/MorseTrainSet_15/GEN15_VER_002/", 'wordsMatrices_15_002', "Words_15_002.csv"],

        ["./training_data/MorseTrainSet_17/GEN17_VER_000/", 'wordsMatrices_17_000', "Words_17_000.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_100/", 'wordsMatrices_17_100', "Words_17_100.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_200/", 'wordsMatrices_17_200', "Words_17_200.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_210/", 'wordsMatrices_17_210', "Words_17_210.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_220/", 'wordsMatrices_17_220', "Words_17_220.csv"],

    ], 
    IMAGE_TARGET_SIZE,
    [position_regression, letter_n_to_index],
    letter_n=LETTER_END_POSITION,
    overwrite_images=False
)

In [ ]:
# Concat label arrays
morse_labels_concat = np.array([morse_labels[0], morse_labels[1]]).T

In [ ]:
# Improve me
def labels_to_one_hot(batch_y_positions, batch_y_letters):
    label_letters = batch_y_letters.astype("int")
    # clean up magic numbers
    labels_one_hot = np.zeros((label_letters.size, CATEGORIES))
    labels_one_hot[np.arange(label_letters.size),label_letters] = 1
    return labels_one_hot


In [ ]:
from morse_helpers import create_all_sets
train, labels, train_validation, labels_validation, train_test, labels_test = create_all_sets(
    image_fnames, morse_labels_concat, TEST_SPLIT_SIZE, VALIDATION_SPLIT_SIZE, shuffle_before_test_split=True)


In [ ]:
from morse_helpers import Image_Generator

training_batch_generator = Image_Generator(train, labels, BATCH_SIZE, IMAGE_TARGET_SIZE, IMAGE_PREPOCESSORS, labels_to_one_hot)
validation_batch_generator = Image_Generator(train_validation, labels_validation, BATCH_SIZE, IMAGE_TARGET_SIZE, IMAGE_PREPOCESSORS, labels_to_one_hot)

In [ ]:
# Display generator data
t, l = training_batch_generator.__getitem__(0)

for img in t:
    fig = plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()

In [ ]:
# Residual block
def get_addblock(x, kernelsize, filters):
    fx = layers.Conv2D(filters, kernelsize, activation='relu', padding='same')(x)
    fx = layers.BatchNormalization()(fx)
    fx = layers.Conv2D(filters, kernelsize, padding='same')(fx)
    out = layers.Add()([x,fx])
    out = layers.ReLU()(out)
    out = layers.BatchNormalization()(out)
    return out

In [ ]:
def conv_model_catg(input_layer):
    
    x = keras.layers.Cropping2D(cropping=((0, 0), (0,1250)), data_format=None)(input_layer)

    x = get_addblock(x, (3,5), 8)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)
    x = get_addblock(x, (3,7), 8)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)
    x = get_addblock(x, (3,3), 8)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)
    x = get_addblock(x, (3,3), 8)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)
  
    x = Flatten()(x)
    
    return x

In [ ]:
from tensorflow.keras import layers

input_layer    = Input(shape=IMAGE_TARGET_SIZE)
conv_model_flattened = conv_model_catg(input_layer)
output_layer_letter    = Dense(CATEGORIES, activation="softmax")(conv_model_flattened)

model = Model(inputs=input_layer, outputs=output_layer_letter)
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=["accuracy"])

print(model.summary())

In [ ]:
init_epoch = 0

In [ ]:

num_epochs = 1

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
					   training_batch_generator,
	                   steps_per_epoch = int(len(train) // BATCH_SIZE),
	                   epochs = epochs + init_epoch,
					   initial_epoch=init_epoch,
	                   verbose =1,
	                   validation_data = validation_batch_generator,
	                   validation_steps = int(len(train_validation) // BATCH_SIZE))
	
	
	init_epoch += epochs
	return history

history = fit_model(num_epochs)


In [ ]:
def show_image(img):
    plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()

In [ ]:
IMAGE_PREPROCESSORS_TEST = [
    {"func" :cut_and_center, "params": ""}, 
    {"func": add_noise_random, "params": [29, 30]}
]

In [ ]:
# Instantiate test generator
test_batch_generator = Image_Generator(train_test, labels_test, BATCH_SIZE, IMAGE_TARGET_SIZE, IMAGE_PREPROCESSORS_TEST, labels_to_one_hot)

In [ ]:
# Display image from test generator

t, l = test_batch_generator.__getitem__(0)
show_image(t[0])

In [ ]:
predictions = model.predict(test_batch_generator)

In [ ]:

def get_deviating_predictions(generator, predictions):
    categorical_differences = []
    indexer = 0

    for imgs_batch, labels_batch in generator:

        for i in range(len(imgs_batch)):

            catg_pred = np.argmax(predictions[indexer])
            catg_test_label = np.argmax(labels_batch[i])

            if catg_pred != catg_test_label:

                categorical_differences.append([catg_pred, catg_test_label, imgs_batch[i], predictions[indexer]])

            indexer += 1

    return categorical_differences


categorical_differences  = get_deviating_predictions(test_batch_generator, predictions)

print("Total label predictions: ", len(predictions))

print("Total label predictions incorrect: ", len(categorical_differences))
print("Label predictions percentage incorrect", round( (len(categorical_differences) / len(predictions) * 100), 4), "%")

In [ ]:
evaluations = model.evaluate(test_batch_generator)
evaluations

In [ ]:
counter = 0   
for diff in categorical_differences:

    counter += 1
    if counter > 30:
        break

    pred, correct, img, one_hot = diff

    print(np.round(one_hot, 2))

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', pred)
    show_image(img_pred)

    print('Correct', correct)
    show_image(img_correct)

    print("----------------------------------------------------------------------------------------")